In [1]:
from UCLSE.environment import Market_session, yamlLoad
from UCLSE.test.utils import identical_replay_vars,side_by_side_period_by_period_difference_checker
import copy

In [2]:
import os
pa=os.getcwd()
config_name='config\\mkt_cfg.yml'
config_path=os.path.join(pa,config_name)
environ_dic=yamlLoad(config_path)

In [3]:
#just updates the tradefile every time there is a trade
environ_dic['dump_each_trade']=True
environ_dic['verbose']=False
environ_dic['end_time']=50

Recreating a market session is vital for refactoring and debugging. Moreover setting the random seed on initialization didn't work when I tried to recreate results. Because the exchange and the traders repeatedly make random actions, recreation is not straightforward.

After extensive rewriting, the simulate method is now separated into a simulate_one_period method which is then repeated for the duration of the experiment. 

simulate_one_period does exactly what you would expect. Within this method, I have separated out the individual tasks that a simulation period undergoes: 

1. Getting supply and demand for the traders
2. Cancel any overwritten orders (still in a one order world)
3. Pick a  limit order from a trader chosen at random
4. Send that order to the exchange for processing.
        This might result in a trade being printed
5. Publish the LOB for traders to update their records
6. (Optional) Record the happenings of the period for recreation 


The simulate method now has the option of recording the events of session stored in attribute  Market_session.replay_vars, so that the session can be identically recreated.

The replay_vars attribute is a dictionary which can be entered as an input into the simulate method for recreation.

Begin by instantiating a Market_session and then copying it exactly.

In [4]:
import copy
sess=Market_session(**environ_dic)
sess1=copy.deepcopy(sess)

Then we can run the first simulation, store the events and push those events into the second simulation session to make sure that the end result is the same.

In [5]:
sess.simulate(sess.trade_stats_df3,recording=True)
sess1.simulate(sess1.trade_stats_df3,replay_vars=sess.replay_vars,recording=True)
(sess1.df.fillna(0)==sess.df.fillna(0)).all() #note that the best_bid or best_bid do not always give a True value. Because nan!=nan.

avg_balance.csv
avg_balance.csv


expid                    True
time                     True
SHVR      balance_sum    True
          n              True
          pc             True
GVWY      balance_sum    True
          n              True
          pc             True
ZIP       balance_sum    True
          n              True
          pc             True
ZIC       balance_sum    True
          n              True
          pc             True
best_bid                 True
best_ask                 True
dtype: bool

This function checks that the two replay_vars attributes are the same and returns the difference if it exists:
    

In [6]:
identical_replay_vars(sess,sess1)

no differences found


{}

For a more forensic approach to difference checking, this function will look for differences at the end of every period and stop if one occurs.

In [7]:
sess=Market_session(**environ_dic)
sess1=copy.deepcopy(sess)
side_by_side_period_by_period_difference_checker(sess,sess1)
(sess1.df==sess.df).all()

avg_balance.csv
avg_balance.csv


expid                    True
time                     True
SHVR      balance_sum    True
          n              True
          pc             True
GVWY      balance_sum    True
          n              True
          pc             True
ZIP       balance_sum    True
          n              True
          pc             True
ZIC       balance_sum    True
          n              True
          pc             True
best_bid                 True
best_ask                 True
dtype: bool

In [8]:
sess1.df

expid     time        SHVR                  GVWY            \
                            balance_sum   n    pc balance_sum   n    pc   
4.6375   trial0001   4.6375         0.0  20  0.00         0.0  20  0.00   
5.6000   trial0001   5.6000         0.0  20  0.00        10.0  20  0.50   
6.1750   trial0001   6.1750        10.0  20  0.50        10.0  20  0.50   
7.4875   trial0001   7.4875        20.0  20  1.00        10.0  20  0.50   
9.1250   trial0001   9.1250        20.0  20  1.00        10.0  20  0.50   
10.0875  trial0001  10.0875        20.0  20  1.00        20.0  20  1.00   
10.9250  trial0001  10.9250        20.0  20  1.00        31.0  20  1.55   
11.0125  trial0001  11.0125        20.0  20  1.00        42.0  20  2.10   
12.1375  trial0001  12.1375        31.0  20  1.55        42.0  20  2.10   
14.7000  trial0001  14.7000        31.0  20  1.55        53.0  20  2.65   
14.7375  trial0001  14.7375        31.0  20  1.55        53.0  20  2.65   
16.7125  trial0001  16.7125        31.0  20  1.55        53.0  20  2.65   
17.0250  trial0001  17.0250        31.0  20  1.55        64.0  20  3.20   
17.7500  trial0001  17.7500        42.0  20  2.10        64.0  20  3.20   
18.8625  trial0001  18.8625        42.0  20  2.10        74.0  20  3.70   
19.3375  trial0001  19.3375        42.0  20  2.10        74.0  20  3.70   
22.2375  trial0001  22.2375        42.0  20  2.10        74.0  20  3.70   
22.5875  trial0001  22.5875        42.0  20  2.10        85.0  20  4.25   
22.8125  trial0001  22.8125        52.0  20  2.60        85.0  20  4.25   
23.2625  trial0001  23.2625        52.0  20  2.60        85.0  20  4.25   
26.1875  trial0001  26.1875        62.0  20  3.10        85.0  20  4.25   
26.9500  trial0001  26.9500        62.0  20  3.10        95.0  20  4.75   
28.5000  trial0001  28.5000        62.0  20  3.10       105.0  20  5.25   
29.2500  trial0001  29.2500        72.0  20  3.60       105.0  20  5.25   
29.3375  trial0001  29.3375        81.0  20  4.05       106.0  20  5.30   
30.5000  trial0001  30.5000        91.0  20  4.55       106.0  20  5.30   
30.9375  trial0001  30.9375        99.0  20  4.95       106.0  20  5.30   
31.1125  trial0001  31.1125       108.0  20  5.40       107.0  20  5.35   
33.0500  trial0001  33.0500       108.0  20  5.40       107.0  20  5.35   
33.1250  trial0001  33.1250       108.0  20  5.40       107.0  20  5.35   
33.9000  trial0001  33.9000       108.0  20  5.40       107.0  20  5.35   
34.8250  trial0001  34.8250       108.0  20  5.40       107.0  20  5.35   
36.1000  trial0001  36.1000       109.0  20  5.45       107.0  20  5.35   
36.2125  trial0001  36.2125       109.0  20  5.45       107.0  20  5.35   
36.2500  trial0001  36.2500       109.0  20  5.45       107.0  20  5.35   
37.1375  trial0001  37.1375       109.0  20  5.45       107.0  20  5.35   
38.1250  trial0001  38.1250       119.0  20  5.95       107.0  20  5.35   
38.1875  trial0001  38.1875       119.0  20  5.95       115.0  20  5.75   
38.9250  trial0001  38.9250       127.0  20  6.35       115.0  20  5.75   
39.3875  trial0001  39.3875       127.0  20  6.35       116.0  20  5.80   
41.0375  trial0001  41.0375       127.0  20  6.35       124.0  20  6.20   
41.6625  trial0001  41.6625       127.0  20  6.35       124.0  20  6.20   
44.2125  trial0001  44.2125       137.0  20  6.85       124.0  20  6.20   
44.4625  trial0001  44.4625       147.0  20  7.35       124.0  20  6.20   
45.4875  trial0001  45.4875       147.0  20  7.35       129.0  20  6.45   
45.5375  trial0001  45.5375       147.0  20  7.35       129.0  20  6.45   
47.4250  trial0001  47.4250       147.0  20  7.35       135.0  20  6.75   
47.5625  trial0001  47.5625       147.0  20  7.35       140.0  20  7.00   
48.9375  trial0001  48.9375       157.0  20  7.85       140.0  20  7.00   
50.0000  trial0001  50.0000       157.0  20  7.85       140.0  20  7.00   

                ZIP                   ZIC           best_bid best_ask  
        balance_sum   n    pc balance_su

In [9]:
sess.df

expid     time        SHVR                  GVWY            \
                            balance_sum   n    pc balance_sum   n    pc   
4.6375   trial0001   4.6375         0.0  20  0.00         0.0  20  0.00   
5.6000   trial0001   5.6000         0.0  20  0.00        10.0  20  0.50   
6.1750   trial0001   6.1750        10.0  20  0.50        10.0  20  0.50   
7.4875   trial0001   7.4875        20.0  20  1.00        10.0  20  0.50   
9.1250   trial0001   9.1250        20.0  20  1.00        10.0  20  0.50   
10.0875  trial0001  10.0875        20.0  20  1.00        20.0  20  1.00   
10.9250  trial0001  10.9250        20.0  20  1.00        31.0  20  1.55   
11.0125  trial0001  11.0125        20.0  20  1.00        42.0  20  2.10   
12.1375  trial0001  12.1375        31.0  20  1.55        42.0  20  2.10   
14.7000  trial0001  14.7000        31.0  20  1.55        53.0  20  2.65   
14.7375  trial0001  14.7375        31.0  20  1.55        53.0  20  2.65   
16.7125  trial0001  16.7125        31.0  20  1.55        53.0  20  2.65   
17.0250  trial0001  17.0250        31.0  20  1.55        64.0  20  3.20   
17.7500  trial0001  17.7500        42.0  20  2.10        64.0  20  3.20   
18.8625  trial0001  18.8625        42.0  20  2.10        74.0  20  3.70   
19.3375  trial0001  19.3375        42.0  20  2.10        74.0  20  3.70   
22.2375  trial0001  22.2375        42.0  20  2.10        74.0  20  3.70   
22.5875  trial0001  22.5875        42.0  20  2.10        85.0  20  4.25   
22.8125  trial0001  22.8125        52.0  20  2.60        85.0  20  4.25   
23.2625  trial0001  23.2625        52.0  20  2.60        85.0  20  4.25   
26.1875  trial0001  26.1875        62.0  20  3.10        85.0  20  4.25   
26.9500  trial0001  26.9500        62.0  20  3.10        95.0  20  4.75   
28.5000  trial0001  28.5000        62.0  20  3.10       105.0  20  5.25   
29.2500  trial0001  29.2500        72.0  20  3.60       105.0  20  5.25   
29.3375  trial0001  29.3375        81.0  20  4.05       106.0  20  5.30   
30.5000  trial0001  30.5000        91.0  20  4.55       106.0  20  5.30   
30.9375  trial0001  30.9375        99.0  20  4.95       106.0  20  5.30   
31.1125  trial0001  31.1125       108.0  20  5.40       107.0  20  5.35   
33.0500  trial0001  33.0500       108.0  20  5.40       107.0  20  5.35   
33.1250  trial0001  33.1250       108.0  20  5.40       107.0  20  5.35   
33.9000  trial0001  33.9000       108.0  20  5.40       107.0  20  5.35   
34.8250  trial0001  34.8250       108.0  20  5.40       107.0  20  5.35   
36.1000  trial0001  36.1000       109.0  20  5.45       107.0  20  5.35   
36.2125  trial0001  36.2125       109.0  20  5.45       107.0  20  5.35   
36.2500  trial0001  36.2500       109.0  20  5.45       107.0  20  5.35   
37.1375  trial0001  37.1375       109.0  20  5.45       107.0  20  5.35   
38.1250  trial0001  38.1250       119.0  20  5.95       107.0  20  5.35   
38.1875  trial0001  38.1875       119.0  20  5.95       115.0  20  5.75   
38.9250  trial0001  38.9250       127.0  20  6.35       115.0  20  5.75   
39.3875  trial0001  39.3875       127.0  20  6.35       116.0  20  5.80   
41.0375  trial0001  41.0375       127.0  20  6.35       124.0  20  6.20   
41.6625  trial0001  41.6625       127.0  20  6.35       124.0  20  6.20   
44.2125  trial0001  44.2125       137.0  20  6.85       124.0  20  6.20   
44.4625  trial0001  44.4625       147.0  20  7.35       124.0  20  6.20   
45.4875  trial0001  45.4875       147.0  20  7.35       129.0  20  6.45   
45.5375  trial0001  45.5375       147.0  20  7.35       129.0  20  6.45   
47.4250  trial0001  47.4250       147.0  20  7.35       135.0  20  6.75   
47.5625  trial0001  47.5625       147.0  20  7.35       140.0  20  7.00   
48.9375  trial0001  48.9375       157.0  20  7.85       140.0  20  7.00   
50.0000  trial0001  50.0000       157.0  20  7.85       140.0  20  7.00   

                ZIP                   ZIC           best_bid best_ask  
        balance_sum   n    pc balance_su